### 1. Importing necessary libraries

In [1]:
import tensorflow as tf
import h5py

2024-03-20 17:30:20.997637: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-20 17:30:22.037337: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### 2. Loading validation dataset

In [2]:
with h5py.File('./output/hdf5/valid.hdf5', 'r') as f:
    validation_data = f['images'][:]
    validation_labels = f['labels'][:]

### 3. Importing the model

In [3]:
from subroutines.loss_metrics import (
    Weighted_Cross_Entropy,
    F1_score as F1_score_metric,
    Precision_dil,
    F1_score_dil,
)

In [4]:
model = tf.keras.models.load_model(
    './output/checkpoints/crack_detection_6_epoch_48_F1_score_0.784.h5',
    custom_objects={
        "loss": Weighted_Cross_Entropy(10),
        "F1_score": F1_score_metric,
        "F1_score_dil": F1_score_dil,
        "Precision_dil": Precision_dil,
    },
)

model.summary()

2024-03-20 17:31:16.331401: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-20 17:31:16.393212: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 225, 225, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1 (Conv2D)              (None, 112, 112, 32)         864       ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 32)         128       ['conv1[0][0]']               
 on)                                                                                          

### 4. Making predictions

In [5]:
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, log_loss

# Predict segmentation masks
predicted_masks = model.predict(validation_data)

# Assuming both validation_labels and predicted_masks are binary masks
validation_labels_flat = validation_labels.flatten()
predicted_masks_flat = (predicted_masks > 0.5).astype(int).flatten()  # Convert probabilities to binary predictions

x = f1_score(validation_labels_flat, predicted_masks_flat)
y = precision_score(validation_labels_flat, predicted_masks_flat)
z = recall_score(validation_labels_flat, predicted_masks_flat)
acc = accuracy_score(validation_labels_flat, predicted_masks_flat)
loss = log_loss(validation_labels_flat, predicted_masks_flat)

2024-03-20 17:31:28.285109: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 256901120 exceeds 10% of free system memory.
2024-03-20 17:31:28.902423: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2024-03-20 17:31:28.925610: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 513802240 exceeds 10% of free system memory.
2024-03-20 17:31:29.632986: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 822083584 exceeds 10% of free system memory.
2024-03-20 17:31:29.687283: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.


2/2 [==============================] - 5s 1s/step


In [6]:
print("F1 Score:", x)
print("Precision Score:", y)
print("Recall Score:", z)
print("Accuracy Score:", acc)
print("Log Loss:", loss)

F1 Score: 0.7983822663122304
Precision Score: 0.7794036630744696
Recall Score: 0.8183082012461186
Accuracy Score: 0.9830891096673045
Log Loss: 0.6095302696530517
